In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127139")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127139
Azure region: southcentralus
Subscription id: a0f586ec-8016-4ea9-8248-9bf2299ad437
Resource group: aml-quickstarts-127139


In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_name = 'aml-cluster'


#To verify that cluster doesn't exist already
try:
    aml_compute = ComputeTarget(workspace = ws, name= compute_name)
    print('Compute Target Found : ' + compute_name)
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes = 4)
    
    #Creating the compute target
    aml_compute = ComputeTarget.create(ws, compute_name, compute_config)

### YOUR CODE HERE ###
aml_compute.wait_for_completion(show_output= True)

Compute Target Found : aml-cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [31]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

project-compute ComputeInstance Succeeded
aml-cluster AmlCompute Succeeded


In [32]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.5, 2), "--max_iter": uniform(10,100)
    }
)
### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor= 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './', 
              compute_target = aml_compute, 
              vm_size= "Standard_D2_V2",
              entry_script= 'train.py')

In [33]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(estimator = est,
                             hyperparameter_sampling =ps,
                             policy = policy,
                             primary_metric_name = "accuracy",
                             primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs= 15,
                             max_concurrent_runs = 3)

In [40]:
#submit hyperdrive experiment
hyperdrive_run= exp.submit(hd_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_16c7fdce-84cb-4f1a-910d-66c07115785c
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_16c7fdce-84cb-4f1a-910d-66c07115785c?wsid=/subscriptions/a0f586ec-8016-4ea9-8248-9bf2299ad437/resourcegroups/aml-quickstarts-127139/workspaces/quick-starts-ws-127139

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-21T12:50:05.908781][API][INFO]Experiment created<END>\n""<START>[2020-11-21T12:50:06.669561][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2020-11-21T12:50:07.010901][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-21T12:50:08.1869747Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [39]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
model = best_run.register_model(model_name='hyperdrive_best_model', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
web_paths = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(path=web_paths)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###